# Section 2 

Start looking in to a more realistic case -- evolving the full set of coefficients


In [1]:
# Boilerplate python imports
import sys
import csv
from math import *
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


In [9]:
# basic1 from the github

# Some global parameters:
nobs = 579
nparameters = 6

npopulation = 10
per_second = 60     # estimate of number of generations per second
genmax = int(60*per_second)

train_start = int(0)
train_end   = int(364)
np.random.seed(0)      # for reproducibility

from evolution1 import *

######################## ######################## ########################
# Now bring in the data for real work:
matchup_set = []

with open('testin.csv') as csvfile:
    k = 0
    sreader = csv.reader(csvfile, delimiter=",")
    for line in sreader:
        day     = float(line[0])
        t2m_gfs = float(line[1])
        td_gfs  = float(line[2])
        thick_gfs = float(line[3])
        rh_gfs  = float(line[4])
        speed   = float(line[5])
        obs_t2m = float(line[6])
        obs_td  = float(line[7])
        terr    = float(line[8])
        tderr  = float(line[9])

        #Note that obs_td, obs_t2m, tderr are being ignored. They can be
        #       added to the list.
        #  n.b.: note that it is terr that is used, not t2m itself.
        #Model and observation are well-enough correlated that it is the increment
        #which makes more sense to predict [Krasnopolsky,20NNN]
        m = matchup((day,t2m_gfs,td_gfs,thick_gfs,rh_gfs,speed,terr))
        matchup_set.append(m)
        k += 1

csvfile.close()
######################## ######################## ########################

Initialize and seed the population

Note the python structure used for initializing and adding to a list of things. Population and bests can be added to at will via the .append operation. We'll use this later (section 3) to collect all the parameter suites which are good in some respect (we'll decide what constitutes 'good').


In [8]:
# basic1 

#Initialize and seed the population
population = []
bests      = []       # Save all then-best versions
for k in range (0,npopulation):
    population.append(critter(nparameters))

weights = np.zeros((nparameters))
sdevs   = np.zeros((nparameters))
bests.append(critter(nparameters))
bests[0].init(weights, sdevs)
nbests = 1

#for reference, take the raw gfs output's score:
population[0].init(weights, sdevs)
score_gfs = population[0].skill(matchup_set, train_start, train_end)

print("uncorrected score in training period: ",
         population[0].skill(matchup_set, train_start, train_end) )
print("uncorrected score in evaluation period: ",
         population[0].skill(matchup_set, train_end+1, nobs), flush=True )
population[0].show_fcst(matchup_set, train_start, train_end)

population[0].weights[0] = -2.0
population[0].show_fcst(matchup_set, train_start, train_end)
population[0].weights[0] = 0.0

print("\n",flush=True)

uncorrected score in training period:  15.619538461538465
uncorrected score in evaluation period:  11.043857868020302
0.0 26.35 23.15 -9.0 78.0 3.8 3.57  zzz  0.0 3.57 False
1.0 26.35 22.65 -8.5 75.0 3.9 3.02  zzz  0.0 3.02 False
2.0 24.95 22.45 -7.6 85.0 7.8 8.84  zzz  0.0 8.84 False
3.0 24.65 22.05 -7.9 83.0 8.1 6.32  zzz  0.0 6.32 False
4.0 24.05 21.85 -7.6 86.0 13.7 5.16  zzz  0.0 5.16 False
5.0 26.85 22.95 -8.7 81.0 21.9 6.85  zzz  0.0 6.85 False
6.0 26.65 23.15 -8.9 78.0 7.9 9.98  zzz  0.0 9.98 False
7.0 26.85 23.25 -8.1 68.0 2.3 11.29  zzz  0.0 11.29 False
8.0 26.85 23.55 -9.4 79.0 5.6 6.29  zzz  0.0 6.29 False
9.0 27.45 23.55 -9.6 76.0 3.6 5.23  zzz  0.0 5.23 False
10.0 28.05 23.85 -9.7 73.0 2.6 8.61  zzz  0.0 8.61 False
11.0 28.55 23.85 -10.0 70.0 2.6 2.99  zzz  0.0 2.99 False
12.0 28.05 22.85 -9.2 70.0 1.7 6.38  zzz  0.0 6.38 False
13.0 27.55 23.25 -8.0 68.0 5.6 4.22  zzz  0.0 4.22 False
14.0 27.75 22.85 -8.7 75.0 2.6 4.97  zzz  0.0 4.97 False
15.0 28.35 23.25 -9.4 74.0 1.7 5

224.0 21.85 19.55 -8.5 78.0 12.4 19.63  zzz  0.0 19.63 False
225.0 18.85 9.65 -4.5 35.0 4.6 17.18  zzz  0.0 17.18 False
226.0 21.95 19.45 -8.2 85.0 8.2 21.39  zzz  0.0 21.39 False
227.0 18.25 15.05 -3.3 62.0 5.2 12.69  zzz  0.0 12.69 False
228.0 17.15 9.75 -11.2 69.0 7.7 16.04  zzz  0.0 16.04 False
229.0 11.75 2.75 -5.4 47.0 7.4 15.64  zzz  0.0 15.64 False
230.0 14.45 4.55 -8.0 47.0 4.8 17.23  zzz  0.0 17.23 False
231.0 11.95 1.95 -3.9 38.0 5.6 15.28  zzz  0.0 15.28 False
232.0 17.05 11.95 -10.4 78.0 8.8 24.27  zzz  0.0 24.27 False
233.0 17.55 13.05 -6.1 58.0 6.6 20.88  zzz  0.0 20.88 False
234.0 18.45 15.15 -9.4 85.0 15.4 27.89  zzz  0.0 27.89 False
235.0 11.45 2.75 -1.1 41.0 5.8 25.89  zzz  0.0 25.89 False
236.0 14.55 2.05 -5.1 33.0 4.4 26.77  zzz  0.0 26.77 False
237.0 12.85 3.05 -4.7 46.0 4.5 25.63  zzz  0.0 25.63 False
238.0 16.25 12.05 -5.9 62.0 6.7 26.81  zzz  0.0 26.81 False
239.0 18.95 14.95 -8.3 78.0 7.0 30.06  zzz  0.0 30.06 False
240.0 19.15 16.85 -7.0 79.0 6.2 26.37  zzz  

73.0 29.25 23.25 -10.0 72.0 2.7 7.58  zzz  -2.0 9.58 False
74.0 28.95 23.05 -10.0 72.0 2.7 10.06  zzz  -2.0 12.06 False
75.0 28.25 22.25 -9.3 70.0 5.0 7.14  zzz  -2.0 9.14 False
76.0 29.45 23.45 -10.8 70.0 5.4 6.67  zzz  -2.0 8.67 False
77.0 29.45 24.25 -10.4 75.0 6.1 4.45  zzz  -2.0 6.45 False
78.0 28.95 23.85 -10.1 78.0 4.8 3.95  zzz  -2.0 5.95 False
79.0 29.15 23.45 -10.8 76.0 5.9 4.15  zzz  -2.0 6.15 False
80.0 28.85 22.95 -10.8 75.0 6.3 1.63  zzz  -2.0 3.63 False
81.0 29.05 24.45 -10.2 78.0 9.4 0.72  zzz  -2.0 2.7199999999999998 False
82.0 29.15 24.55 -10.4 77.0 8.5 0.82  zzz  -2.0 2.82 False
83.0 28.65 23.35 -10.6 74.0 4.8 1.98  zzz  -2.0 3.98 False
84.0 27.65 23.15 -9.5 77.0 3.8 3.21  zzz  -2.0 5.21 False
85.0 27.75 22.95 -9.5 73.0 5.6 2.75  zzz  -2.0 4.75 False
86.0 28.15 23.15 -11.1 77.0 6.1 -1.29  zzz  -2.0 0.71 True
87.0 27.75 22.05 -11.2 75.0 4.5 -1.69  zzz  -2.0 0.31000000000000005 True
88.0 28.55 20.95 -12.0 68.0 4.2 2.44  zzz  -2.0 4.4399999999999995 False
89.0 28.75 21.

245.0 20.85 18.25 -7.9 66.0 6.3 21.96  zzz  -2.0 23.96 False
246.0 20.95 17.45 -8.6 65.0 6.4 20.39  zzz  -2.0 22.39 False
247.0 21.55 18.45 -8.2 69.0 2.0 27.11  zzz  -2.0 29.11 False
248.0 22.65 18.85 -8.4 69.0 10.3 29.87  zzz  -2.0 31.87 False
249.0 21.75 19.75 -7.1 81.0 4.8 27.31  zzz  -2.0 29.31 False
250.0 19.95 16.85 -5.4 91.0 4.0 28.28  zzz  -2.0 30.28 False
251.0 19.15 16.35 -4.1 83.0 2.0 28.59  zzz  -2.0 30.59 False
252.0 16.95 14.75 -2.5 87.0 3.7 29.17  zzz  -2.0 31.17 False
253.0 18.95 11.75 -7.5 57.0 4.7 27.84  zzz  -2.0 29.84 False
254.0 19.25 14.55 -6.6 55.0 1.0 27.03  zzz  -2.0 29.03 False
255.0 19.95 15.45 -5.9 46.0 1.1 27.73  zzz  -2.0 29.73 False
256.0 22.05 17.75 -9.7 69.0 14.7 34.27  zzz  -2.0 36.27 False
257.0 14.25 9.25 -4.7 61.0 11.0 21.47  zzz  -2.0 23.47 False
258.0 15.65 9.05 -4.5 50.0 5.5 21.21  zzz  -2.0 23.21 False
259.0 14.85 6.65 -4.1 27.0 7.5 19.29  zzz  -2.0 21.29 False
260.0 17.35 6.85 -4.8 26.0 3.0 24.57  zzz  -2.0 26.57 False
261.0 19.25 10.95 -7.8 28

Initialize the population and find our first best. 

In [4]:
#Initializing the standard deviations for evolution ----------
#For the bias
sdevs[0] = 1.0
#For linear terms
for k in range (1,int(6)):
    sdevs[k] = 1.0

#For quadratic terms
#for k in range (int(6), nparameters):
#  sdevs[k] = 0.0125

#Initialize the population itself now -------------------------
for k in range (0,npopulation):
  weights[0] = np.random.normal(0,sdevs[0])
  for l in range (1, int(6) ):     #initialize only the linear part
    weights[l] = np.random.normal(0,sdevs[l])
  population[k].init(weights,sdevs)

#recall that the matchup_set is holding the matchups
#Find our first 'best' -- noting that we aren't saving raw gfs as an example
smin = 9999.
kbest = int(npopulation)
for k in range (0,npopulation):
    population[k].skill(matchup_set, train_start, train_end)
    if (population[k].score < smin):
        kbest = k
        smin = population[k].score

#Start accumulating our best critters
bests.append(critter(nparameters))
bests[nbests].init(population[kbest].weights, population[kbest].sdevs)
nbests += 1

population[kbest].show()
print("initial kbest, smin = ",kbest, smin, flush=True)


0 0.9500884175255894 1.0
1 -0.1513572082976979 1.0
2 -0.10321885179355784 1.0
3 0.41059850193837233 1.0
4 0.144043571160878 1.0
5 1.454273506962975 1.0
initial kbest, smin =  1 9.775825339684086


For this evolution, we are using only mutation -- as would happen with bacteria (haploid).

As an analogy to diploids (plants, animals, people), one could also have 'crossover' mutations. Namely, to select two parents and take the first M genes from the first parent, and the remainder from the second. 

In [5]:
######################## ######################## ########################
#      Now carry out the (mutation-only) evolution
#swap best in to all slots
#then evolve a new raft of critters from that
#evaluate them
#repeat until limit of generations or happy

for gen in range(0,genmax):
    #print("generation ", gen, flush=True)

    population[0].copy(population[kbest])
    population[0].score = population[kbest].score
    score_best = float(population[0].score)
    smin = score_best
    kbest = 0
    for k in range (1, npopulation):
        population[k].copy(population[0])
        population[k].evolve()
        population[k].skill(matchup_set, train_start, train_end)
        if (population[k].score < score_best):
            kbest = k
            smin = population[k].score
            bests.append(critter(nparameters))
            bests[nbests].init(population[kbest].weights, population[kbest].sdevs)
            nbests += 1
    if (kbest != 0):
        if (score_gfs != 0):
          print("new best ",gen, kbest, smin, score_best, smin/score_gfs, flush=True)
        else:
          print("new best ",gen, kbest, smin, score_best, flush=True)
        population[kbest].show()


new best  2 8 9.18612267636147 9.775825339684086 0.5881174209456552
0 0.5667513171053311 2.0106481828332865
1 -0.5476984668883029 1.0037780078337095
2 -0.7830473750545375 2.5391982310016687
3 -1.1741428077506328 1.4048983955008632
4 0.4201216631199747 0.9844402124417343
5 0.28774992541344924 1.174600592224003
new best  7 4 8.99856052530168 9.18612267636147 0.576109245958818
0 0.3008453533481656 0.49676963405803537
1 -0.5411681711223134 0.25704555014951447
2 -0.6991945341402486 0.07265118793785233
3 -1.0807479954877337 0.5414053400004487
4 0.4243006564176645 0.19841368640833443
5 0.34947730887724393 0.620659800543008
new best  24 6 8.915433328045856 8.99856052530168 0.5707872450904493
0 1.2938766377537154 1.2224427223428291
1 0.08071736381312078 1.1384176498319518
2 -1.7097175147179318 0.9202484815143807
3 -0.7337689771944809 0.6773405339892832
4 0.49154709750139935 0.7833644975385049
5 -0.011123149348269856 0.7300687592962771
new best  124 9 8.704159177795512 8.915433328045856 0.557260

Now consider what we found along the way

In [6]:
######################## ######################## ########################
if (score_gfs != 0):
  print("best score in training period ",gen, kbest, smin, score_best, smin/score_gfs, flush=True)
else:
  print("best score in training period ",gen, kbest, smin, score_best, flush=True)
print("score in the untrained period: ",population[kbest].skill(matchup_set, train_end+1, nobs))

print("found ",nbests,"new bests along the way\n")
for k in range (0, nbests):
  bests[k].show()
  print("\n")



best score in training period  3599 0 7.9351120592361255 7.9351120592361255 0.508024745979245
score in the untrained period:  5.917958099935862
found  13 new bests along the way

0 0.0 0.0
1 0.0 0.0
2 0.0 0.0
3 0.0 0.0
4 0.0 0.0
5 0.0 0.0


0 0.9500884175255894 1.0
1 -0.1513572082976979 1.0
2 -0.10321885179355784 1.0
3 0.41059850193837233 1.0
4 0.144043571160878 1.0
5 1.454273506962975 1.0


0 0.5667513171053311 2.0106481828332865
1 -0.5476984668883029 1.0037780078337095
2 -0.7830473750545375 2.5391982310016687
3 -1.1741428077506328 1.4048983955008632
4 0.4201216631199747 0.9844402124417343
5 0.28774992541344924 1.174600592224003


0 0.3008453533481656 0.49676963405803537
1 -0.5411681711223134 0.25704555014951447
2 -0.6991945341402486 0.07265118793785233
3 -1.0807479954877337 0.5414053400004487
4 0.4243006564176645 0.19841368640833443
5 0.34947730887724393 0.620659800543008


0 1.2938766377537154 1.2224427223428291
1 0.08071736381312078 1.1384176498319518
2 -1.7097175147179318 0.920248

In [7]:
print("Forecasts in the training period:")
population[0].show_fcst(matchup_set, train_start, train_end)
print("Untrained forecasts:")
population[0].show_fcst(matchup_set, train_end, nobs)


Forecasts in the training period:
0.0 26.35 23.15 -9.0 78.0 3.8 3.57  zzz  13.027270104872372 -9.457270104872372 False
1.0 26.35 22.65 -8.5 75.0 3.9 3.02  zzz  12.457754694686097 -9.437754694686097 False
2.0 24.95 22.45 -7.6 85.0 7.8 8.84  zzz  15.373705285066437 -6.533705285066437 True
3.0 24.65 22.05 -7.9 83.0 8.1 6.32  zzz  15.187956023182927 -8.867956023182927 False
4.0 24.05 21.85 -7.6 86.0 13.7 5.16  zzz  15.686429840628467 -10.526429840628467 False
5.0 26.85 22.95 -8.7 81.0 21.9 6.85  zzz  11.847554257021901 -4.997554257021902 True
6.0 26.65 23.15 -8.9 78.0 7.9 9.98  zzz  12.469231383683699 -2.4892313836836983 True
7.0 26.85 23.25 -8.1 68.0 2.3 11.29  zzz  9.74872676412773 1.5412732358722696 True
8.0 26.85 23.55 -9.4 79.0 5.6 6.29  zzz  12.738383932559719 -6.448383932559719 False
9.0 27.45 23.55 -9.6 76.0 3.6 5.23  zzz  11.977752928094752 -6.7477529280947515 False
10.0 28.05 23.85 -9.7 73.0 2.6 8.61  zzz  10.799045080772554 -2.189045080772555 True
11.0 28.55 23.85 -10.0 70.0 2.6

123.0 27.95 22.45 -10.6 78.0 10.7 8.51  zzz  12.873996264928904 -4.363996264928904 True
124.0 27.95 22.15 -10.7 75.0 5.0 7.95  zzz  12.917923929558427 -4.9679239295584265 True
125.0 27.45 21.25 -10.7 70.0 4.5 3.56  zzz  12.338112133952311 -8.77811213395231 False
126.0 27.65 21.35 -11.0 74.0 5.8 3.21  zzz  13.359030670268137 -10.149030670268136 False
127.0 27.25 21.95 -9.9 73.0 2.4 6.14  zzz  12.765639812876401 -6.625639812876401 False
128.0 27.15 23.65 -9.0 85.0 8.5 10.48  zzz  14.053977603257575 -3.5739776032575747 True
129.0 26.45 22.05 -9.4 81.0 6.7 12.56  zzz  14.700042362422273 -2.140042362422273 True
130.0 26.15 20.75 -9.2 74.0 5.9 10.59  zzz  13.837671488592616 -3.2476714885926157 True
131.0 26.35 18.35 -11.1 61.0 6.4 9.68  zzz  12.258563066467051 -2.5785630664670514 True
132.0 26.15 18.65 -11.0 63.0 3.8 10.59  zzz  12.930039124458455 -2.340039124458455 True
133.0 26.15 18.35 -11.2 65.0 3.7 8.93  zzz  13.882894161532828 -4.952894161532829 True
134.0 26.15 16.95 -12.5 60.0 5.5 5.

336.0 22.45 18.25 -6.7 79.0 6.3 11.89  zzz  17.856448926800095 -5.966448926800094 True
337.0 24.45 15.45 -10.6 46.0 7.8 11.12  zzz  10.336067999490234 0.7839320005097647 True
338.0 25.25 15.85 -9.8 44.0 5.1 13.58  zzz  9.390420257387845 4.189579742612155 True
339.0 25.45 17.35 -9.1 49.0 5.3 13.23  zzz  9.371321670427642 3.858678329572358 True
340.0 26.05 18.85 -9.7 58.0 9.8 12.72  zzz  10.249186352125049 2.4708136478749516 True
341.0 26.65 19.25 -9.6 49.0 10.5 14.98  zzz  6.862351228532432 8.117648771467568 True
342.0 26.55 20.05 -9.1 58.0 9.4 -2.34  zzz  8.988286131964236 -11.328286131964235 False
343.0 25.85 21.65 -8.8 64.0 5.9 4.74  zzz  9.848391529205793 -5.108391529205793 False
344.0 26.55 20.15 -9.7 65.0 7.7 7.11  zzz  11.391867515916541 -4.281867515916541 True
345.0 26.95 20.35 -10.8 69.0 10.0 3.06  zzz  12.306633889810723 -9.246633889810722 False
346.0 26.95 21.35 -11.3 74.0 11.0 9.17  zzz  12.901697746931756 -3.731697746931756 True
347.0 26.55 22.05 -10.4 73.0 10.6 5.99  zzz  

441.0 29.65 23.15 -9.7 74.0 4.1 7.98  zzz  11.326299832377613 -3.3462998323776123 True
442.0 29.75 23.35 -9.6 69.0 1.4 4.19  zzz  9.866180272700818 -5.676180272700818 False
443.0 30.05 23.25 -9.2 65.0 2.1 6.16  zzz  8.506552365646481 -2.346552365646481 True
444.0 29.75 22.65 -8.9 62.0 2.4 5.31  zzz  8.09037979529266 -2.780379795292661 False
445.0 30.35 23.15 -9.8 62.0 2.1 10.35  zzz  7.718345586127162 2.6316544138728375 False
446.0 30.85 23.25 -11.4 64.0 3.4 4.18  zzz  8.292516774150828 -4.112516774150828 False
447.0 30.85 23.25 -11.5 66.0 4.0 4.74  zzz  8.863276270980512 -4.1232762709805115 False
448.0 30.95 23.95 -11.6 66.0 4.3 6.51  zzz  8.173649028304983 -1.6636490283049827 True
449.0 30.65 24.05 -11.7 66.0 5.8 6.21  zzz  7.966485245030298 -1.756485245030298 True
450.0 29.75 23.85 -11.1 77.0 12.6 5.86  zzz  10.818339709241641 -4.958339709241641 False
451.0 30.25 22.75 -12.5 70.0 10.9 3.03  zzz  10.026404233481342 -6.996404233481343 False
452.0 29.65 21.85 -11.5 66.0 4.7 7.43  zzz  